In [2]:
import optimizer # optimizer.py file

import analysis # visualization 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import os 
import sys 

import warnings
warnings.filterwarnings("ignore")

In [36]:
# We define the timesteps in the APP as 15 minute 
delta_t = 0.25 #hour 
print("For delta_t: ",delta_t, "max number of intervals:",24/delta_t)
################## Define the TOU Cost ##########################################
## the TOU cost is defined considering the delta_t above, if not code raises an error.##

# off-peak 0.175  cents / kwh 
TOU_tariff = np.ones((96,)) * 17.5
## 4 pm - 9 pm peak 0.367 cents / kwh 
TOU_tariff[64:84] = 36.7
## 9 am - 2 pm super off-peak 0.49 $ / kWh  to cents / kwh
TOU_tariff[36:56] = 14.9


par = optimizer .Parameters(z0 = np.array([20,20,1,1]).reshape(4,1),   #due to local optimality, we can try multiple starts
                            Ts = delta_t,
                            eff = 1.0 ,
                            soft_v_eta = 1e-4,
                            opt_eps = 0.0001,
                            TOU = TOU_tariff) 

###### Define users problem ######

# we receive signals as hours from the app
arrival_hour = 7.25
duration_hour = 24


event = {
        "time" : int(arrival_hour / delta_t),  # INTERVAL 
        "e_need": 10.5, # FLOAT 
        "duration" :int(duration_hour / delta_t), # the slider allows 15 min increments INT
        "station_pow_max"   : 6.6, 
        "user_power_rate":100
        }


print(event['time'])
print(event['duration'])
print(event['e_need'])


prb = optimizer .Problem(par = par, event = event)

opt = optimizer .Optimization(par, prb)

res = opt .run_opt()


For delta_t:  0.25 max number of intervals: 96.0
28
96
10.5


/Users/aysetugbaozturk/Documents/eCal/SlrpEV/pev-operation-hil/pev-operation-hil/scripts/Python Code/optimizer.py:425: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return  np.array([J_1 , J_2 , J_3])
/Users/aysetugbaozturk/miniconda3/lib/python3.8/site-packages/cvxpy/expressions/expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
/Users/aysetugbaozturk/miniconda3/lib/python3.8/site-packages/cvxpy/interface/numpy_interfac

13 -0.33475956685987285


In [37]:
print("SCH Tariff (cents/hr):", np.round(res['z_hr'][0]) ,"\nREG Tariff (cents/hr):" ,np.round(res['z_hr'][1]))

SCH Tariff (cents/hr): 134.0 
REG Tariff (cents/hr): 143.0


/Users/aysetugbaozturk/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
# plot_power_energy(df_plot, prb, test, start_hour,start_minute,flex_end_hour ,flex_end_minute):
# analysis.record_results(event, par, prb, opt, res, 1, vis = True, result_data=False)

In [21]:
def calculate_probability(z_reg, z_sch):
    u_reg =  0.3411 -0.0184*(z_reg - z_sch)*.5
    u_sch = 0.0184*(z_reg - z_sch)*.5
    u_leave = -1. + 0.005*(np.mean([z_sch, z_reg]))

    
    denom = np.exp(u_reg) + np.exp(u_sch) + np.exp(u_leave)

    p_reg = np.exp(u_reg) / denom
    p_sch = np.exp(u_sch) / denom
    p_leave = np.exp(u_leave) / denom
    print(u_reg, u_sch, u_leave)
    return p_reg, p_sch, p_leave 

In [16]:
calculate_probability(143, 134)

0.25830000000000003 0.0828 -0.3075


(0.41546475523589854, 0.3485904668504385, 0.23594477791366306)

In [17]:
res['J']

array([ -8.749125  , -30.63677031, -31.85000587, -32.93886046,
       -34.42583779, -35.14246509, -35.8510344 , -36.43343339,
       -36.88993195, -37.35224462, -37.68844942, -38.02990502,
       -38.37604445])